In [59]:
import pandas as pd
from collections import Counter 
from sklearn.preprocessing import scale
import numpy as np
np.random.seed(42)

In [60]:
'''
ft:挑选后的特征
t:特征对应金标准(无)
name:样本名称
c: 特征对应簇
'''
df1 = pd.read_csv('./feats_bank_train_1111.csv',header = None)
# print(df1.shape)
ft = df1.values #特征样本/128维
# ft = scale(ft)

df2 = pd.read_csv('./train_1111.txt',sep = ' ',header = None)#一般无需修改
name = df2.iloc[:,0].values #图片位置名称
# t = df2.iloc[:,1].values #金标准0/1
# print('Original dataset:', Counter(t))

df3 = pd.read_csv('./kmeans/RA005_VAE.csv',header = 0)
c = df3.iloc[:,1].values
print('Clustering:', Counter(c))

Clustering: Counter({6: 7052, 3: 6452, 0: 6151, 2: 5622, 4: 5541, 7: 5221, 8: 3611, 9: 2019, 5: 1460, 1: 801})


In [61]:
n_clusters = 10
ratio = 0.50


index=[] #index[i]表示第i个聚类对应的index列表
ft_part=[] #ft_part[i]表示第i个聚类对应的特征向量列表
for i in range(n_clusters):
    index.append(np.where(c == i)[0])
    ft_part.append(ft[index[i]])

In [62]:
from modAL.density import information_density

new_index=[]
equal_per_cluster = True#False

for i in range(n_clusters):
#     print("Part %d start..."%i)
    cosine_density = information_density(ft_part[i], 'cosine')
#     euclidean_density = information_density(ft_part[i], 'euclidean')

    if equal_per_cluster:
        sample_per_cluster=int(len(ft)*ratio*1.0/n_clusters)
#         sample_per_cluster = 1
    else:
        sample_per_cluster=int(ratio*1.0*len(ft_part[i]))
    
    li=np.argsort(-cosine_density)[:sample_per_cluster]#1310/2620/3930/5240
#     li=np.argsort(-euclidean_density)[:295]
    new_index.extend(index[i][li])
#     print("Part %d finished."%i)
    
# new_index = np.squeeze(new_index)
# new_index = new_index.flatten()#根据代表性计算后的index，一维的list

ft_sel = ft[new_index]
# t_sel = t[new_index]
c_sel = c[new_index]
name_sel = name[new_index]

print('Resampled dataset:', Counter(c_sel))
len(c_sel)
# print(len(t_sel))
# a=dict(Counter(t_sel))
# np.mean(list(a.values())), np.var(list(a.values()))
#maybe euclidean的信息样本的效果更好
#cosine_density的非信息样本更好一些

Resampled dataset: Counter({0: 2196, 2: 2196, 3: 2196, 4: 2196, 6: 2196, 7: 2196, 8: 2196, 9: 2019, 5: 1460, 1: 801})


19652

In [63]:
'''
注意保存csv的后缀
'''
# print(__doc__)
# dataframe1 = pd.DataFrame({'image_name':name_sel,'cluster':c_sel})
# dataframe1.to_csv("../txt/RA30.txt",index=False,sep=' ',header = None)
# print(Counter(c_sel))
print(__doc__)
dataframe1 = pd.DataFrame({'image_name':name_sel})
dataframe1.to_csv("./RA_pick_1126/MAE_R50_RA.txt",index=False,sep=',',header = None)


注意保存csv的后缀



In [64]:
# dataframe1 = pd.DataFrame(ft_sel)
# dataframe1.to_csv("./RA_pick_1126/MAE_R5_RA.txt",index=False,sep=',',header = None)